In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import seaborn as sns

import sys
sys.path.append('../')
from facematch import Facematch
import tqdm
from deepface import DeepFace
%load_ext autotime


time: 401 µs (started: 2024-02-07 15:58:50 -03:00)


In [2]:
df = pd.read_parquet('../base/filtered_scored_data.parquet')

time: 20.8 ms (started: 2024-02-07 15:58:50 -03:00)


In [11]:
samples_size = 10

df2use = df[df['approved'] == True].sample(samples_size, random_state=42)
df2use = pd.concat([df2use, df[df['approved'] == False].sample(samples_size, random_state=42)])
#df2use = df2use.sample(samples_size, random_state=42)

time: 4.16 ms (started: 2024-02-07 16:10:04 -03:00)


In [12]:
registration_images = df2use.registration_image.values
comparision_image = df2use.comparision_image.values
unico_status = df2use.approved.values

for i in tqdm.tqdm(range(0, len(df2use))):
    fig, ax = plt.subplots(1, 2, figsize=(10, 10))

    img_0 = cv2.imread('../base/imagens/' + registration_images[i])
    img_1 = cv2.imread('../base/imagens/' + comparision_image[i])

    ax[0].imshow(img_0[:,:,::-1])
    ax[1].imshow(img_1[:,:,::-1])

    similarity = Facematch.verify(img_0, img_1).get('similarity')
    deepface_val = 1 - DeepFace.verify(img_0, img_1, model_name='Facenet512', enforce_detection=False, detector_backend='mediapipe')['distance']

    string = f"Image {i}, unico : {unico_status[i]}, similarity: {similarity}, deepface: {round(deepface_val,2)}"

    ax[0].set_title(string)
    plt.savefig('images_2_check/' + str(i) + '.png')
    plt.close()

100%|██████████| 20/20 [00:46<00:00,  2.33s/it]

time: 46.7 s (started: 2024-02-07 16:10:04 -03:00)


In [10]:
len(df2use)

20

time: 2.05 ms (started: 2024-02-07 16:10:02 -03:00)
